In [ ]:
!pip install transformers
!pip install --upgrade s3fs
!pip uninstall botocore -y
!pip install botocore
!pip install --upgrade boto3
!pip install tensorflow-hub
!pip install tensorflow-text

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
from sklearn.metrics import roc_auc_score
import s3fs

from transformers import DistilBertTokenizer
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig, DistilBertTokenizerFast
from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import pickle
import boto3
import io
import os

import tempfile
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model

2023-12-11 01:44:08.326208: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 01:44:08.326249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 01:44:08.327392: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# set seed
seed_value = 42
tf.random.set_seed(seed_value)

### Load Data

In [4]:
# export these datasets to s3 as csv files
s3_path = 's3://fake-job-posting-detection/Data'
train = pd.read_csv(f'{s3_path}/train.csv')
train_2x = pd.read_csv(f'{s3_path}/augmented_train_2x.csv')
train_10x = pd.read_csv(f'{s3_path}/augmented_train_10x.csv')
test = pd.read_csv(f'{s3_path}/test.csv')
validation = pd.read_csv(f'{s3_path}/validation.csv')




In [6]:
# prepare input and output data
X_train = train['text']
y_train = train['fraudulent']
X_train_2x = train_2x['text']
y_train_2x = train_2x['fraudulent']
X_train_10x = train_10x['text']
y_train_10x = train_10x['fraudulent']
X_valid = validation['text']
y_valid = validation['fraudulent']
X_test = test['text']
y_test = test['fraudulent']

### Tokenize Data

In [7]:
# Initialize the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

max_length=512

# Tokenize the training, validation, and testing input data
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=max_length)
train_2x_encodings = tokenizer(X_train_2x.tolist(), truncation=True, padding=True, max_length=max_length)
train_10x_encodings = tokenizer(X_train_10x.tolist(), truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(X_valid.tolist(), truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=max_length)

# Convert target labels to TensorFlow tensors
train_labels = tf.convert_to_tensor(y_train)
train_labels_2x = tf.convert_to_tensor(y_train_2x)
train_labels_10x = tf.convert_to_tensor(y_train_10x)
val_labels = tf.convert_to_tensor(y_valid)
test_labels = tf.convert_to_tensor(y_test)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [9]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [10]:
from transformers import TFAutoModel

def create_model():
    
    # initialize the BERT model
    bert = TFAutoModel.from_pretrained('bert-base-uncased')

    # two input layers, layer name variables MUST match the dictionary keys 
    input_ids = tf.keras.layers.Input(shape=(max_length,), name='input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(max_length,), name='attention_mask', dtype='int32')

    # we access the transformer model within our bert object using the bert attribute 
    embeddings = bert.bert(input_ids, attention_mask=mask)[1]  # access final activations (alread max-pooled) [1]
    
    
    # convert bert embeddings into 2 output classes
    x = tf.keras.layers.Dense(256, activation='relu')(embeddings)

    # Add a dropout layer
    dropout_rate = 0.3
    x = tf.keras.layers.Dropout(rate=dropout_rate)(x)
    
    # add the output layer
    y = tf.keras.layers.Dense(1, activation='sigmoid', name='outputs')(x)

    # initialize model
    model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

    # freeze bert layer
    model.layers[2].trainable = False

    # print out model summary
    return model

# create a model
model = create_model()
model.summary()

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109482240 (417.64 MB)
Trainable params: 109482240 (417.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 512)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 512)]                0         []                            
 )                       

In [11]:
batch_size=128


# Assuming train_encodings and val_encodings are BatchEncoding objects
train_dataset = tf.data.Dataset.from_tensor_slices((
    {key: train_encodings[key] for key in ['input_ids', 'attention_mask']},
    train_labels  # Assuming train_labels is your training labels
)).shuffle(100).batch(batch_size)

# Assuming train_encodings and val_encodings are BatchEncoding objects
train_dataset_2x = tf.data.Dataset.from_tensor_slices((
    {key: train_2x_encodings[key] for key in ['input_ids', 'attention_mask']},
    train_labels_2x  # Assuming train_labels is your training labels
)).shuffle(100).batch(batch_size)


# Assuming train_encodings and val_encodings are BatchEncoding objects
train_dataset_10x = tf.data.Dataset.from_tensor_slices((
    {key: train_10x_encodings[key] for key in ['input_ids', 'attention_mask']},
    train_labels_10x  # Assuming train_labels is your training labels
)).shuffle(100).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {key: val_encodings[key] for key in ['input_ids', 'attention_mask']},
    val_labels  # Assuming val_labels is your validation labels
)).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {key: test_encodings[key] for key in ['input_ids', 'attention_mask']},
    test_labels  # Assuming val_labels is your validation labels
)).batch(batch_size)


### Training Without Data Augmentation

In [ ]:

num_epochs=30

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-5, decay=1e-6)
loss = 'binary_crossentropy'  # Use binary crossentropy for binary classification
auc = tf.keras.metrics.AUC()  # Use AUC as the metric for binary classification

# compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=[auc])

# Define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True)

# Then use these datasets to train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=num_epochs, callbacks=[early_stopping])

In [15]:
# evluate the dataset
results = model.evaluate(test_dataset)


25/25 [==============================] - 117s 5s/step - loss: 0.1659 - auc: 0.8001


In [16]:
# Print evaluation results
print("Test Loss:", results[0])  # Prints the test loss
print("Test AUC:", results[1])  # Prints the test accuracy (if accuracy was specified as a metric)

Test Loss: 0.165871262550354
Test AUC: 0.8001484274864197


In [25]:

# set path for model
s3_bucket_name = 'fake-job-posting-detection'
s3_key = 'Model/model.h5'

# Save the model to a temporary file locally
with tempfile.NamedTemporaryFile(suffix='.h5', delete=False) as temp_file:
    model.save(temp_file.name)

# Create a connection to S3 using boto3
s3_client = boto3.client('s3')

# Upload the temporary model file to S3
s3_client.upload_file(temp_file.name, s3_bucket_name, s3_key)

# Remove the temporary file after upload
temp_file.close()

### Training with data augmentation (2x)

In [87]:
model_2x = create_model()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "tf_bert_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109482240 (417.64 MB)
Trainable params: 109482240 (417.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [88]:
num_epochs=30

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-5, decay=1e-6)
loss = 'binary_crossentropy'  # Use binary crossentropy for binary classification
auc = tf.keras.metrics.AUC()  # Use AUC as the metric for binary classification

# compile the model
model_2x.compile(optimizer=optimizer, loss=loss, metrics=[auc])

# Define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True)

# Then use these datasets to train the model
model_2x.fit(train_dataset_2x, validation_data=val_dataset, epochs=num_epochs, callbacks=[early_stopping])

Epoch 1/30
44/44 [==============================] - 320s 7s/step - loss: 0.4214 - auc_2: 0.4026 - val_loss: 0.2017 - val_auc_2: 0.4945
Epoch 2/30
44/44 [==============================] - 308s 7s/step - loss: 0.3448 - auc_2: 0.4312 - val_loss: 0.1915 - val_auc_2: 0.5298
Epoch 3/30
44/44 [==============================] - 308s 7s/step - loss: 0.3472 - auc_2: 0.4009 - val_loss: 0.1884 - val_auc_2: 0.5809
Epoch 4/30
44/44 [==============================] - 307s 7s/step - loss: 0.3366 - auc_2: 0.4419 - val_loss: 0.1873 - val_auc_2: 0.6059
Epoch 5/30
44/44 [==============================] - 308s 7s/step - loss: 0.3416 - auc_2: 0.4071 - val_loss: 0.1864 - val_auc_2: 0.6184
Epoch 6/30
44/44 [==============================] - 308s 7s/step - loss: 0.3314 - auc_2: 0.4495 - val_loss: 0.1858 - val_auc_2: 0.6521
Epoch 7/30
44/44 [==============================] - 308s 7s/step - loss: 0.3295 - auc_2: 0.4619 - val_loss: 0.1857 - val_auc_2: 0.6522
Epoch 8/30
44/44 [==============================] - 307

In [91]:


s3_bucket_name = 'fake-job-posting-detection'
s3_key = 'Model/model_2x_augmentation.h5'

# Save the model to a temporary file locally
with tempfile.NamedTemporaryFile(suffix='.h5', delete=False) as temp_file:
    model_2x.save(temp_file.name)

# Create a connection to S3 using boto3
s3_client = boto3.client('s3')

# Upload the temporary model file to S3
s3_client.upload_file(temp_file.name, s3_bucket_name, s3_key)

# Remove the temporary file after upload
temp_file.close()

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Training with data augmentation (10x)

In [89]:
model_10x = create_model()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "tf_bert_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109482240 (417.64 MB)
Trainable params: 109482240 (417.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [90]:
num_epochs=30

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-5, decay=1e-6)
loss = 'binary_crossentropy'  # Use binary crossentropy for binary classification
auc = tf.keras.metrics.AUC()  # Use AUC as the metric for binary classification

# compile the model
model_10x.compile(optimizer=optimizer, loss=loss, metrics=[auc])

# Define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True)

# Then use these datasets to train the model
model_10x.fit(train_dataset_10x, validation_data=val_dataset, epochs=num_epochs, callbacks=[early_stopping])

Epoch 1/30
44/44 [==============================] - 320s 7s/step - loss: 0.4118 - auc_3: 0.3965 - val_loss: 0.1977 - val_auc_3: 0.4631
Epoch 2/30
44/44 [==============================] - 308s 7s/step - loss: 0.3370 - auc_3: 0.4121 - val_loss: 0.1924 - val_auc_3: 0.4852
Epoch 3/30
44/44 [==============================] - 307s 7s/step - loss: 0.3402 - auc_3: 0.3873 - val_loss: 0.1889 - val_auc_3: 0.5535
Epoch 4/30
44/44 [==============================] - 307s 7s/step - loss: 0.3341 - auc_3: 0.4104 - val_loss: 0.1886 - val_auc_3: 0.6104
Epoch 5/30
44/44 [==============================] - 307s 7s/step - loss: 0.3331 - auc_3: 0.4166 - val_loss: 0.1879 - val_auc_3: 0.6222
Epoch 6/30
44/44 [==============================] - 307s 7s/step - loss: 0.3307 - auc_3: 0.4256 - val_loss: 0.1877 - val_auc_3: 0.6288
Epoch 7/30
44/44 [==============================] - 307s 7s/step - loss: 0.3282 - auc_3: 0.4356 - val_loss: 0.1874 - val_auc_3: 0.6367
Epoch 8/30
44/44 [==============================] - 307

In [92]:
# file path
s3_bucket_name = 'fake-job-posting-detection'
s3_key = 'Model/model_10x_augmentation.h5'

# Save the model to a temporary file locally
with tempfile.NamedTemporaryFile(suffix='.h5', delete=False) as temp_file:
    model_10x.save(temp_file.name)

# Create a connection to S3 using boto3
s3_client = boto3.client('s3')

# Upload the temporary model file to S3
s3_client.upload_file(temp_file.name, s3_bucket_name, s3_key)

# Remove the temporary file after upload
temp_file.close()